<a href="https://colab.research.google.com/github/ingabLee/Book_GenerateAIForComptureVision/blob/main/GenerateAI_Vision_Chapter_2_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch
!pip install pytorch-model-summary

In [4]:
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import pytorch_model_summary

from google.colab import drive

drive.mount('/content/drive')

dirWork = '/content/drive/MyDrive/VisionForGenerateAI/Chapter2'
dirImage = dirWork + '/VAE_img'

os.makedirs(dirImage, exist_ok=True)

Mounted at /content/drive


In [5]:
# tensor data normalization. (find min, max value)
def normalization(tensor, min_value, max_value):
  min_tensor = tensor.min()
  tensor = (tensor - min_tensor)
  max_tensor = tensor.max()
  tensor = tensor / max_tensor
  tensor = tensor * (max_value - min_value) + min_value
  return tensor

# tensor value round
def value_round(tensor):
  return torch.round(tensor)

# 생성한 tensor을 이미지로 출력하기 위한 차원변환을 수행하는 함수.
def to_img(x):
  x = x.view(x.size(0), 1, 28, 28)
  return x

# MINST 데이터를 학습에 사용하기위해서 전처리를 수행하는 transform함수를 선언
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:value_round(tensor))
])

batch_size = 128    # 배치 사이즈 128

# load MNIST dataset with transform(전처리)
dataset = MNIST(dirWork + '/MNIST_dataset', transform=img_transform, download=True)

# 배치사이즈에 맞게 dataloader를 설정하여 shuffle옵션을 통해 섞는다.
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class VariationalAutoencoder(nn.Module):
  def __init__(self):
    super(VariationalAutoencoder, self).__init__()
    self.encoder = nn.Sequential(
        nn.Linear(28 * 28, 400)

    )